# 🏠 Smart Home Temperature Digital Twin
## Build Your First Digital Twin in 60 Minutes!

**What we'll build**: A Digital Twin of a smart home temperature system that predicts when the AC needs maintenance.

**Real-world scenario**: Your home has temperature sensors. We'll create a virtual copy that predicts problems before they happen!

---

### 📚 What You'll Learn
1. ✅ Load real IoT sensor data from Kaggle
2. ✅ Clean and visualize the data
3. ✅ Build a simple prediction model (ML)
4. ✅ Create a Digital Twin class in Python
5. ✅ Run real-time simulations
6. ✅ Visualize with live dashboard

### ⏱️ Time Required
- **Total**: 60 minutes
- **Setup**: 5 minutes
- **Coding**: 45 minutes
- **Experimentation**: 10 minutes

### 🎯 Prerequisites
- Basic Python knowledge (variables, loops, functions)
- Curiosity! 😊

---

## 🚀 Let's Start!

---
# STEP 1: Setup & Import Libraries (5 minutes)
Let's install everything we need and import the libraries.

In [ ]:
# Install required packages (only needed first time)
!pip install -q pandas numpy matplotlib seaborn scikit-learn plotly kaggle

print("✅ All packages installed successfully!")

In [ ]:
# Import all libraries we'll use
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import time
import warnings
warnings.filterwarnings('ignore')

# For machine learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Set style for pretty plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ All libraries imported!")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🔢 NumPy version: {np.__version__}")

---
# STEP 2: Load Real IoT Temperature Data (5 minutes)

We'll use a real dataset of IoT temperature sensors. This simulates data from a smart home.

**Dataset**: IoT Temperature Monitoring
- **What it contains**: Room temperature, humidity, AC status
- **Size**: ~5000 readings over time
- **Goal**: Predict when AC might fail

In [ ]:
# For this workshop, we'll create synthetic data that mimics real IoT sensors
# In production, this would come from actual sensors!

np.random.seed(42)  # For reproducible results

# Generate 5000 time points (like 5000 sensor readings)
n_samples = 5000

# Create realistic temperature data
# Normal AC operation: 20-24°C, Failing AC: 25-30°C
normal_temp = np.random.normal(22, 1.5, int(n_samples * 0.7))  # 70% normal
failing_temp = np.random.normal(27, 2, int(n_samples * 0.3))    # 30% failing

# Combine and shuffle
temperature = np.concatenate([normal_temp, failing_temp])
np.random.shuffle(temperature)

# Create humidity data (correlated with temperature)
humidity = 50 + (temperature - 22) * 2 + np.random.normal(0, 3, n_samples)

# AC power consumption (watts)
power_consumption = 800 + (temperature - 22) * 50 + np.random.normal(0, 50, n_samples)

# Operating hours (how long AC has been running)
operating_hours = np.random.randint(0, 10000, n_samples)

# Room occupancy (number of people)
occupancy = np.random.randint(0, 5, n_samples)

# Target: AC Status (0 = Normal, 1 = Needs Maintenance)
# Rule: If temp > 25 OR power > 1000, needs maintenance
ac_status = ((temperature > 25) | (power_consumption > 1000)).astype(int)

# Create DataFrame
df = pd.DataFrame({
    'timestamp': pd.date_range(start='2024-01-01', periods=n_samples, freq='5min'),
    'temperature': temperature,
    'humidity': humidity,
    'power_consumption': power_consumption,
    'operating_hours': operating_hours,
    'occupancy': occupancy,
    'ac_status': ac_status
})

print("✅ Dataset created!")
print(f"\n📊 Dataset shape: {df.shape[0]} readings, {df.shape[1]} features")
print(f"\n🔍 First 5 readings:")
df.head()

### 🎓 Understanding the Data

**Each row** = One sensor reading (every 5 minutes)

**Columns explained**:
- `timestamp`: When the reading was taken
- `temperature`: Room temperature in °C
- `humidity`: Air humidity in %
- `power_consumption`: AC power usage in watts
- `operating_hours`: Total hours AC has been running
- `occupancy`: Number of people in the room
- `ac_status`: **TARGET** - Does AC need maintenance? (0=No, 1=Yes)

In [ ]:
# Let's explore the data
print("📈 DATA SUMMARY")
print("=" * 50)
print(df.describe().round(2))

print("\n🔍 MISSING VALUES CHECK")
print("=" * 50)
print(df.isnull().sum())

print("\n⚙️ AC STATUS DISTRIBUTION")
print("=" * 50)
status_counts = df['ac_status'].value_counts()
print(f"Normal (0): {status_counts[0]} readings ({status_counts[0]/len(df)*100:.1f}%)")
print(f"Needs Maintenance (1): {status_counts[1]} readings ({status_counts[1]/len(df)*100:.1f}%)")

---
# STEP 3: Visualize the Data (10 minutes)

Let's see what our sensor data looks like!

In [ ]:
# Create a beautiful dashboard of our data
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('🏠 Smart Home IoT Sensor Data Analysis', fontsize=16, fontweight='bold')

# 1. Temperature over time
axes[0, 0].plot(df['timestamp'][:500], df['temperature'][:500], linewidth=1, alpha=0.7)
axes[0, 0].set_title('Temperature Over Time (First 500 readings)', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Time')
axes[0, 0].set_ylabel('Temperature (°C)')
axes[0, 0].axhline(y=25, color='r', linestyle='--', label='Danger Threshold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Temperature distribution by AC status
df[df['ac_status']==0]['temperature'].hist(ax=axes[0, 1], bins=30, alpha=0.5, label='Normal', color='green')
df[df['ac_status']==1]['temperature'].hist(ax=axes[0, 1], bins=30, alpha=0.5, label='Needs Maintenance', color='red')
axes[0, 1].set_title('Temperature Distribution by AC Status', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Temperature (°C)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Correlation heatmap
corr = df[['temperature', 'humidity', 'power_consumption', 'operating_hours', 'occupancy']].corr()
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', ax=axes[1, 0], cbar_kws={'label': 'Correlation'})
axes[1, 0].set_title('Feature Correlations', fontsize=12, fontweight='bold')

# 4. Power vs Temperature scatter
scatter = axes[1, 1].scatter(df['temperature'], df['power_consumption'], 
                             c=df['ac_status'], cmap='RdYlGn_r', alpha=0.5, s=20)
axes[1, 1].set_title('Power Consumption vs Temperature', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Temperature (°C)')
axes[1, 1].set_ylabel('Power Consumption (W)')
cbar = plt.colorbar(scatter, ax=axes[1, 1])
cbar.set_label('AC Status (0=Normal, 1=Maintenance)')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Visualizations complete!")
print("\n💡 Key Observations:")
print("   - Temperature above 25°C usually means AC needs maintenance")
print("   - Higher power consumption correlates with higher temperature")
print("   - We can predict AC failures before they happen!")

---
# STEP 4: Feature Engineering (5 minutes)

Let's create new features that help our model predict better!

**Feature Engineering** = Creating new data columns from existing ones

In [ ]:
# Create rolling averages (trend over last hour = 12 readings)
df['temp_rolling_mean'] = df['temperature'].rolling(window=12, min_periods=1).mean()
df['temp_rolling_std'] = df['temperature'].rolling(window=12, min_periods=1).std().fillna(0)

# Rate of temperature change
df['temp_change'] = df['temperature'].diff().fillna(0)

# Humidity-temperature interaction
df['temp_humidity_ratio'] = df['temperature'] / (df['humidity'] + 0.1)  # Avoid division by zero

# Power efficiency (power per degree)
df['power_efficiency'] = df['power_consumption'] / (df['temperature'] + 0.1)

# Is temperature above normal threshold?
df['temp_alarm'] = (df['temperature'] > 24).astype(int)

# Is power consumption high?
df['power_alarm'] = (df['power_consumption'] > 950).astype(int)

print("✅ New features created!")
print(f"\n📊 Dataset now has {df.shape[1]} columns (features)")
print("\nNew features:")
new_features = ['temp_rolling_mean', 'temp_rolling_std', 'temp_change', 
                'temp_humidity_ratio', 'power_efficiency', 'temp_alarm', 'power_alarm']
for i, feat in enumerate(new_features, 1):
    print(f"   {i}. {feat}")

df.head()

---
# STEP 5: Train Machine Learning Model (10 minutes)

Now let's teach a computer to predict AC failures!

**What we're doing**: Training a "Random Forest" model - it's like creating a team of decision trees that vote on the answer.

In [ ]:
# Select features for training
feature_columns = [
    'temperature', 'humidity', 'power_consumption', 'operating_hours', 'occupancy',
    'temp_rolling_mean', 'temp_rolling_std', 'temp_change',
    'temp_humidity_ratio', 'power_efficiency', 'temp_alarm', 'power_alarm'
]

# Prepare data
X = df[feature_columns]  # Input features
y = df['ac_status']      # What we want to predict

# Split into training (80%) and testing (20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("✅ Data prepared for training!")
print(f"\n📊 Training set: {len(X_train)} samples")
print(f"📊 Testing set: {len(X_test)} samples")
print(f"\n🎯 We're predicting: ac_status (0=Normal, 1=Needs Maintenance)")

In [ ]:
# Train the model
print("🤖 Training Random Forest model...\n")

model = RandomForestClassifier(
    n_estimators=100,      # Number of trees
    max_depth=10,          # How deep each tree grows
    min_samples_split=10,  # Minimum samples to split a node
    random_state=42,
    n_jobs=-1              # Use all CPU cores
)

start_time = time.time()
model.fit(X_train, y_train)
training_time = time.time() - start_time

print(f"✅ Model trained in {training_time:.2f} seconds!")
print(f"\n🌳 Created {model.n_estimators} decision trees")
print(f"🎯 Ready to make predictions!")

In [ ]:
# Make predictions
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probability of maintenance needed

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print("📊 MODEL PERFORMANCE")
print("=" * 60)
print(f"\n✅ Accuracy: {accuracy:.2%}")
print(f"   (Out of 100 predictions, {int(accuracy*100)} are correct!)\n")

# Detailed report
print("\n📋 Detailed Classification Report:")
print("=" * 60)
print(classification_report(y_test, y_pred, target_names=['Normal (0)', 'Maintenance (1)']))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix: AC Status Predictions', fontsize=14, fontweight='bold')
plt.ylabel('Actual Status')
plt.xlabel('Predicted Status')
plt.xticks([0.5, 1.5], ['Normal', 'Maintenance'])
plt.yticks([0.5, 1.5], ['Normal', 'Maintenance'])
plt.show()

print("\n💡 How to read this:")
print(f"   ✅ Top-left ({cm[0,0]}): Correctly predicted Normal")
print(f"   ✅ Bottom-right ({cm[1,1]}): Correctly predicted Maintenance")
print(f"   ❌ Top-right ({cm[0,1]}): False Alarms (predicted Maintenance, was Normal)")
print(f"   ❌ Bottom-left ({cm[1,0]}): Missed Failures (predicted Normal, was Maintenance)")

In [ ]:
# Feature Importance - What matters most?
feature_importance = pd.DataFrame({
    'Feature': feature_columns,
    'Importance': model.feature_importances_
}).sort_values('Importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance, x='Importance', y='Feature', palette='viridis')
plt.title('🔍 Which Features Matter Most for Predictions?', fontsize=14, fontweight='bold')
plt.xlabel('Importance Score')
plt.tight_layout()
plt.show()

print("\n🏆 Top 3 Most Important Features:")
for idx, row in feature_importance.head(3).iterrows():
    print(f"   {row['Feature']:.<30} {row['Importance']:.4f}")

---
# STEP 6: Build the Digital Twin! 🎉 (15 minutes)

This is where the magic happens! We'll create a Python class that represents our AC system.

**Digital Twin** = A virtual copy of our AC that:
1. Receives sensor data
2. Predicts failures using our ML model
3. Tracks health over time
4. Generates alerts

In [ ]:
class SmartHomeDigitalTwin:
    """
    Digital Twin of a Smart Home AC System
    
    This virtual AC mirrors the real one and predicts problems!
    """
    
    def __init__(self, model, feature_columns):
        """
        Initialize the Digital Twin
        
        Args:
            model: Trained ML model
            feature_columns: List of feature names
        """
        self.model = model
        self.feature_columns = feature_columns
        
        # Current state of the AC system
        self.state = {
            'temperature': 22.0,
            'humidity': 50.0,
            'power_consumption': 800.0,
            'operating_hours': 0,
            'occupancy': 0,
            'health_score': 100.0,
            'failure_probability': 0.0,
            'status': 'NORMAL',
            'alerts': []
        }
        
        # Historical data
        self.history = []
        
        # For rolling calculations
        self.recent_temps = [22.0] * 12  # Last 12 readings
        
    def update_sensors(self, sensor_data):
        """
        Receive new sensor readings from the physical AC
        
        Args:
            sensor_data: Dict with sensor readings
        """
        # Update state with new sensor readings
        self.state['temperature'] = sensor_data.get('temperature', self.state['temperature'])
        self.state['humidity'] = sensor_data.get('humidity', self.state['humidity'])
        self.state['power_consumption'] = sensor_data.get('power_consumption', self.state['power_consumption'])
        self.state['occupancy'] = sensor_data.get('occupancy', self.state['occupancy'])
        self.state['operating_hours'] += 1
        
        # Update recent temperatures for rolling calculations
        self.recent_temps.append(self.state['temperature'])
        self.recent_temps.pop(0)  # Remove oldest
        
        # Calculate derived features
        self._calculate_features()
        
        # Predict failure probability using ML model
        self._predict_failure()
        
        # Check for alerts
        self._check_alerts()
        
        # Update health score
        self._update_health_score()
        
        # Save to history
        self.history.append({
            'timestamp': datetime.now(),
            **self.state
        })
    
    def _calculate_features(self):
        """Calculate engineered features"""
        self.state['temp_rolling_mean'] = np.mean(self.recent_temps)
        self.state['temp_rolling_std'] = np.std(self.recent_temps)
        self.state['temp_change'] = self.recent_temps[-1] - self.recent_temps[-2]
        self.state['temp_humidity_ratio'] = self.state['temperature'] / (self.state['humidity'] + 0.1)
        self.state['power_efficiency'] = self.state['power_consumption'] / (self.state['temperature'] + 0.1)
        self.state['temp_alarm'] = 1 if self.state['temperature'] > 24 else 0
        self.state['power_alarm'] = 1 if self.state['power_consumption'] > 950 else 0
    
    def _predict_failure(self):
        """Use ML model to predict failure probability"""
        # Prepare features in correct order
        features = np.array([[
            self.state['temperature'],
            self.state['humidity'],
            self.state['power_consumption'],
            self.state['operating_hours'],
            self.state['occupancy'],
            self.state['temp_rolling_mean'],
            self.state['temp_rolling_std'],
            self.state['temp_change'],
            self.state['temp_humidity_ratio'],
            self.state['power_efficiency'],
            self.state['temp_alarm'],
            self.state['power_alarm']
        ]])
        
        # Get probability of failure
        proba = self.model.predict_proba(features)[0][1]
        self.state['failure_probability'] = proba
        
        # Update status based on probability
        if proba > 0.7:
            self.state['status'] = '🔴 CRITICAL'
        elif proba > 0.4:
            self.state['status'] = '🟡 WARNING'
        else:
            self.state['status'] = '🟢 NORMAL'
    
    def _check_alerts(self):
        """Generate maintenance alerts"""
        self.state['alerts'] = []
        
        if self.state['temperature'] > 26:
            self.state['alerts'].append('⚠️ HIGH TEMPERATURE')
        
        if self.state['power_consumption'] > 1000:
            self.state['alerts'].append('⚠️ HIGH POWER CONSUMPTION')
        
        if self.state['failure_probability'] > 0.7:
            self.state['alerts'].append('🚨 MAINTENANCE REQUIRED SOON')
        
        if self.state['operating_hours'] > 8000:
            self.state['alerts'].append('🔧 SCHEDULE ROUTINE MAINTENANCE')
    
    def _update_health_score(self):
        """Calculate overall health score (0-100)"""
        # Health decreases as failure probability increases
        self.state['health_score'] = (1 - self.state['failure_probability']) * 100
    
    def get_dashboard(self):
        """Return dashboard-ready data"""
        return {
            '🏠 System Status': self.state['status'],
            '💚 Health Score': f"{self.state['health_score']:.1f}%",
            '⚠️ Failure Risk': f"{self.state['failure_probability']:.1%}",
            '🌡️ Temperature': f"{self.state['temperature']:.1f}°C",
            '💧 Humidity': f"{self.state['humidity']:.1f}%",
            '⚡ Power': f"{self.state['power_consumption']:.0f}W",
            '⏱️ Operating Hours': f"{self.state['operating_hours']:,}h",
            '👥 Occupancy': f"{self.state['occupancy']} people",
            '🔔 Active Alerts': ', '.join(self.state['alerts']) if self.state['alerts'] else '✅ None'
        }

print("✅ Digital Twin class created!")
print("\n🎉 Our virtual AC system is ready to run!")

---
# STEP 7: Run the Digital Twin Simulation! 🚀 (10 minutes)

Let's see our Digital Twin in action with real-time updates!

In [ ]:
# Create our Digital Twin
twin = SmartHomeDigitalTwin(model, feature_columns)

print("🏠 SMART HOME DIGITAL TWIN - LIVE SIMULATION")
print("=" * 80)
print("\nSimulating 20 sensor readings (in real life, these come from actual sensors)\n")

# Simulate 20 time steps
for i in range(20):
    # Simulate sensor readings (in production, this comes from IoT devices)
    # Let's simulate a gradual temperature rise (AC starting to fail)
    base_temp = 22 + (i * 0.3)  # Temperature rises slowly
    
    sensor_data = {
        'temperature': base_temp + np.random.normal(0, 0.5),
        'humidity': 50 + (base_temp - 22) * 2 + np.random.normal(0, 2),
        'power_consumption': 800 + (base_temp - 22) * 40 + np.random.normal(0, 30),
        'occupancy': np.random.randint(0, 4)
    }
    
    # Update the Digital Twin
    twin.update_sensors(sensor_data)
    
    # Get dashboard view
    dashboard = twin.get_dashboard()
    
    # Print update
    print(f"\n⏰ Update #{i+1} | {datetime.now().strftime('%H:%M:%S')}")
    print("-" * 80)
    for key, value in dashboard.items():
        print(f"   {key:<20} {value}")
    
    # Simulate time passing (remove in production)
    time.sleep(0.5)

print("\n" + "=" * 80)
print("✅ Simulation complete!")
print(f"\n📊 Collected {len(twin.history)} data points in our Digital Twin")

---
# STEP 8: Visualize Digital Twin Dashboard 📊 (5 minutes)

Let's create an interactive dashboard to monitor our Digital Twin!

In [ ]:
# Convert history to DataFrame for easy plotting
history_df = pd.DataFrame(twin.history)

# Create interactive dashboard with Plotly
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=(
        '🌡️ Temperature Over Time',
        '💚 Health Score',
        '⚡ Power Consumption',
        '⚠️ Failure Probability',
        '💧 Humidity',
        '📊 Current Status'
    ),
    specs=[
        [{"type": "scatter"}, {"type": "scatter"}],
        [{"type": "scatter"}, {"type": "scatter"}],
        [{"type": "scatter"}, {"type": "indicator"}]
    ]
)

# Temperature
fig.add_trace(
    go.Scatter(y=history_df['temperature'], mode='lines+markers', 
               name='Temperature', line=dict(color='red', width=2)),
    row=1, col=1
)
fig.add_hline(y=25, line_dash="dash", line_color="orange", row=1, col=1)

# Health Score
fig.add_trace(
    go.Scatter(y=history_df['health_score'], mode='lines+markers',
               name='Health', line=dict(color='green', width=2),
               fill='tozeroy'),
    row=1, col=2
)

# Power Consumption
fig.add_trace(
    go.Scatter(y=history_df['power_consumption'], mode='lines+markers',
               name='Power', line=dict(color='blue', width=2)),
    row=2, col=1
)

# Failure Probability
fig.add_trace(
    go.Scatter(y=history_df['failure_probability'], mode='lines+markers',
               name='Risk', line=dict(color='orange', width=2)),
    row=2, col=2
)
fig.add_hline(y=0.7, line_dash="dash", line_color="red", row=2, col=2)

# Humidity
fig.add_trace(
    go.Scatter(y=history_df['humidity'], mode='lines+markers',
               name='Humidity', line=dict(color='cyan', width=2)),
    row=3, col=1
)

# Current Health Gauge
current_health = twin.state['health_score']
gauge_color = "green" if current_health > 70 else "orange" if current_health > 40 else "red"

fig.add_trace(
    go.Indicator(
        mode="gauge+number",
        value=current_health,
        title={'text': "Current Health"},
        gauge={
            'axis': {'range': [0, 100]},
            'bar': {'color': gauge_color},
            'steps': [
                {'range': [0, 40], 'color': "lightgray"},
                {'range': [40, 70], 'color': "lightgray"},
                {'range': [70, 100], 'color': "lightgray"}
            ],
            'threshold': {
                'line': {'color': "red", 'width': 4},
                'thickness': 0.75,
                'value': 70
            }
        }
    ),
    row=3, col=2
)

# Update layout
fig.update_layout(
    height=900,
    showlegend=False,
    title_text="🏠 Smart Home Digital Twin - Live Dashboard",
    title_font_size=20
)

fig.update_xaxes(title_text="Time Step", row=3, col=1)
fig.update_yaxes(title_text="°C", row=1, col=1)
fig.update_yaxes(title_text="Score %", row=1, col=2)
fig.update_yaxes(title_text="Watts", row=2, col=1)
fig.update_yaxes(title_text="Probability", row=2, col=2)
fig.update_yaxes(title_text="%", row=3, col=1)

fig.show()

print("✅ Interactive dashboard created!")
print("\n💡 Try hovering over the charts to see values!")

---
# STEP 9: Test "What-If" Scenarios 🔮 (5 minutes)

The power of Digital Twins: We can test scenarios without touching the real AC!

In [ ]:
print("🔮 WHAT-IF SCENARIO TESTING")
print("=" * 80)
print("\nLet's test: What happens if temperature suddenly spikes?\n")

# Create a fresh Digital Twin for testing
test_twin = SmartHomeDigitalTwin(model, feature_columns)

# Scenario 1: Normal operation
print("📊 Scenario 1: Normal Operation (22°C)")
test_twin.update_sensors({
    'temperature': 22,
    'humidity': 50,
    'power_consumption': 800,
    'occupancy': 2
})
dashboard1 = test_twin.get_dashboard()
print(f"   Status: {dashboard1['🏠 System Status']}")
print(f"   Health: {dashboard1['💚 Health Score']}")
print(f"   Risk: {dashboard1['⚠️ Failure Risk']}")

# Scenario 2: Temperature spike
print("\n📊 Scenario 2: Temperature Spike (28°C)")
test_twin.update_sensors({
    'temperature': 28,
    'humidity': 60,
    'power_consumption': 1100,
    'occupancy': 4
})
dashboard2 = test_twin.get_dashboard()
print(f"   Status: {dashboard2['🏠 System Status']}")
print(f"   Health: {dashboard2['💚 Health Score']}")
print(f"   Risk: {dashboard2['⚠️ Failure Risk']}")
print(f"   Alerts: {dashboard2['🔔 Active Alerts']}")

# Scenario 3: After maintenance
print("\n📊 Scenario 3: After Maintenance (20°C, low power)")
test_twin2 = SmartHomeDigitalTwin(model, feature_columns)
test_twin2.update_sensors({
    'temperature': 20,
    'humidity': 45,
    'power_consumption': 750,
    'occupancy': 1
})
dashboard3 = test_twin2.get_dashboard()
print(f"   Status: {dashboard3['🏠 System Status']}")
print(f"   Health: {dashboard3['💚 Health Score']}")
print(f"   Risk: {dashboard3['⚠️ Failure Risk']}")

print("\n" + "=" * 80)
print("✅ What-if testing complete!")
print("\n💡 Key Insight: We can predict problems BEFORE they happen in real life!")

---
# 🎉 CONGRATULATIONS! You Built a Digital Twin!

## 🏆 What You've Accomplished

✅ Loaded and explored IoT sensor data  
✅ Visualized temperature and power patterns  
✅ Created predictive features (feature engineering)  
✅ Trained a machine learning model (Random Forest)  
✅ Built a complete Digital Twin class in Python  
✅ Ran real-time simulations  
✅ Created an interactive dashboard  
✅ Tested "what-if" scenarios  

## 🚀 Next Steps

### Beginner Challenges
1. **Experiment with thresholds**: Change the temperature threshold from 25°C to 23°C
2. **Add new features**: Create a feature for "time since last maintenance"
3. **Try different visualizations**: Add a pie chart for status distribution

### Intermediate Challenges
1. **Compare models**: Try `XGBoost` instead of Random Forest
2. **Add more sensors**: Include outdoor temperature, time of day
3. **Create alerts**: Send email when health drops below 50%

### Advanced Challenges
1. **Deploy to web**: Use Streamlit to create a web dashboard
2. **Real-time data**: Connect to actual IoT sensors (Raspberry Pi)
3. **Optimization**: Find optimal temperature to minimize cost

## 📚 Learn More

**Kaggle Datasets to Try**:
- [HVAC Energy Prediction](https://www.kaggle.com/datasets/claytonmiller/hvac-energy)
- [Smart Home Dataset](https://www.kaggle.com/datasets/taranvee/smart-home-dataset-with-weather-information)
- [Predictive Maintenance](https://www.kaggle.com/datasets/shivamb/machine-predictive-maintenance-classification)

**Resources**:
- [Scikit-learn Documentation](https://scikit-learn.org/)
- [Plotly Tutorials](https://plotly.com/python/)
- [Digital Twin Guide](https://www.ibm.com/topics/what-is-a-digital-twin)

## 💬 Share Your Work!

1. **Save this notebook**: File → Download → Download .ipynb
2. **Upload to GitHub**: Share your customized version
3. **Tag us**: #DigitalTwinWorkshop

---

## 🎓 Certificate-Worthy Skills

You now know how to:
- Load and process IoT data
- Engineer features for machine learning
- Train and evaluate ML models
- Build object-oriented Python applications
- Create interactive visualizations
- Implement Digital Twin architecture

**You're officially a Digital Twin Developer! 🎉**

---

### 📧 Questions?
- Email: workshop@digitaltwin.example.com
- Discord: [Join our community](https://discord.gg/digitaltwin)
- LinkedIn: #DigitalTwinLearning

### ⭐ Rate This Workshop
Found this helpful? Give us feedback!
- ⭐⭐⭐⭐⭐ Excellent
- ⭐⭐⭐⭐ Good
- ⭐⭐⭐ Average
- ⭐⭐ Needs Improvement
- ⭐ Start Over

---

**Remember**: Every expert was once a beginner. Keep learning, keep building! 🚀

---
# 🔧 BONUS: Save Your Model & Twin

Let's save everything so you can use it later!

In [ ]:
import joblib
import json

# Save the trained model
joblib.dump(model, 'smart_home_model.pkl')
print("✅ Model saved as 'smart_home_model.pkl'")

# Save feature columns
with open('feature_columns.json', 'w') as f:
    json.dump(feature_columns, f)
print("✅ Feature columns saved")

# Save Digital Twin history
history_df.to_csv('digital_twin_history.csv', index=False)
print("✅ Digital Twin history saved")

print("\n🎉 Everything saved! You can now:")
print("   1. Load the model: model = joblib.load('smart_home_model.pkl')")
print("   2. Load features: with open('feature_columns.json') as f: features = json.load(f)")
print("   3. View history: pd.read_csv('digital_twin_history.csv')")

---
# 📝 BONUS: Quiz - Test Your Knowledge!

Answer these questions to reinforce your learning:

1. **What is a Digital Twin?**
   - A) A backup of computer files
   - B) A virtual copy of a physical system
   - C) A robot that looks human
   - D) A cloud storage service

2. **What does our Random Forest model predict?**
   - A) Tomorrow's temperature
   - B) Whether AC needs maintenance
   - C) Electricity bill
   - D) Room occupancy

3. **What is feature engineering?**
   - A) Building bridges
   - B) Creating new data columns from existing ones
   - C) Fixing bugs in code
   - D) Hardware manufacturing

4. **Why is Digital Twin useful?**
   - A) Save money on repairs
   - B) Predict problems before they happen
   - C) Test scenarios safely
   - D) All of the above ✅

5. **What library did we use for machine learning?**
   - A) TensorFlow
   - B) scikit-learn ✅
   - C) PyTorch
   - D) Keras

**Answers**: 1-B, 2-B, 3-B, 4-D, 5-B

**Your Score**: ___ / 5

- **5/5**: Digital Twin Expert! 🏆
- **4/5**: Almost There! 🌟
- **3/5**: Good Start! 👍
- **<3/5**: Review the notebook again! 📚